In [189]:
# imports
import numpy as np
import pandas as pd
import os
import requests
import foursquare
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [201]:
# import langitude and latitude coordinates from Part 1 City bike acitivity and store in a Dataframe
city_bike_coord = pd.read_csv('../data/CityBike_longlat.csv')
city_bike_coord = city_bike_coord.round(2)
city_bike_coord

,longitude,latitude
0,-79.88,43.26
1,-79.87,43.27
2,-79.87,43.27
3,-79.87,43.26
4,-79.87,43.26
...,...,...
139,-79.87,43.26
140,-79.91,43.26
141,-79.87,43.25
142,-79.91,43.26


In [209]:
# get each pair of long lat coordinates
long_lat_strings = []

for index, row in city_bike_coord.iterrows():
  # Extract the longitude and latitude values
  longitude = row['longitude']
  latitude = row['latitude']

  # Combine them into a tuple
  long_lat_pair = f"{longitude},{latitude}"

  long_lat_strings.append(long_lat_pair)

long_lat_strings

['-79.88,43.26',
 '-79.87,43.27',
 '-79.87,43.27',
 '-79.87,43.26',
 '-79.87,43.26',
 '-79.88,43.25',
 '-79.87,43.25',
 '-79.88,43.26',
 '-79.88,43.26',
 '-79.86,43.26',
 '-79.92,43.26',
 '-79.83,43.25',
 '-79.88,43.25',
 '-79.88,43.26',
 '-79.84,43.25',
 '-79.83,43.25',
 '-79.88,43.26',
 '-79.88,43.27',
 '-79.88,43.25',
 '-79.89,43.26',
 '-79.89,43.26',
 '-79.89,43.25',
 '-79.92,43.26',
 '-79.92,43.26',
 '-79.92,43.26',
 '-79.92,43.26',
 '-79.9,43.27',
 '-79.9,43.26',
 '-79.91,43.25',
 '-79.9,43.26',
 '-79.9,43.26',
 '-79.89,43.25',
 '-79.89,43.25',
 '-79.95,43.27',
 '-79.89,43.27',
 '-79.88,43.26',
 '-79.92,43.25',
 '-79.9,43.26',
 '-79.92,43.25',
 '-79.91,43.26',
 '-79.91,43.26',
 '-79.93,43.26',
 '-79.91,43.26',
 '-79.92,43.25',
 '-79.91,43.26',
 '-79.95,43.27',
 '-79.87,43.25',
 '-79.91,43.26',
 '-79.87,43.25',
 '-79.83,43.25',
 '-79.93,43.26',
 '-79.94,43.25',
 '-79.94,43.26',
 '-79.83,43.26',
 '-79.93,43.26',
 '-79.94,43.25',
 '-79.87,43.26',
 '-79.89,43.26',
 '-79.87,43.26',
 '

In [214]:

# API URL and API token stored as an environment variable
client_access_token = os.environ['FOURSQUARE_AUTH_TOKEN']

# get response in json while passing API token in the header
headers = {
    "Accept": "application/json",
    "Authorization": client_access_token
}

responses = []

for long_lat_string in long_lat_strings[0:10]:
    #URL
    
    longitude, latitude = long_lat_string.split(",")
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius=1000&query=restaurant"
    response = requests.get(url, headers=headers)
    data = response.json()
    responses.append(data)

print(responses)


[{'results': [{'fsq_id': '4c57494d30d82d7fd583d962', 'categories': [{'id': 13298, 'name': 'Lebanese Restaurant', 'short_name': 'Lebanese', 'plural_name': 'Lebanese Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_', 'suffix': '.png'}}, {'id': 13302, 'name': 'Mediterranean Restaurant', 'short_name': 'Mediterranean', 'plural_name': 'Mediterranean Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_', 'suffix': '.png'}}, {'id': 13303, 'name': 'Mexican Restaurant', 'short_name': 'Mexican', 'plural_name': 'Mexican Restaurants', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}}], 'chains': [], 'closed_bucket': 'LikelyOpen', 'distance': 135, 'geocodes': {'main': {'latitude': 43.259414, 'longitude': -79.878417}, 'roof': {'latitude': 43.259414, 'longitude': -79.878417}}, 'link': '/v3/places/4c57494d30d82d7fd583d962', 'location': {'address': '306 King St W', 'country': '

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [215]:
# create a new dataframe we will append to
results =[]


#iterate through results "data['results]" will give iterate through json data from one level down
for result in data['results']:
    name = result['name']
    location = result['location']

    categories = []
    for category in result['categories']:
        categories.append(category['name'])

    result_dict = {
        'name': name,
        'location': location,
        'categories': categories
    }
    # add respective POI in list object
    results.append(result_dict)

In [216]:
results

[{'name': 'Bonanza Bakery',
  'location': {'address': '100 Murray St E',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '100 Murray St E, Hamilton ON L8L 3G4',
   'locality': 'Hamilton',
   'postcode': 'L8L 3G4',
   'region': 'ON'},
  'categories': ['Bakery']},
 {'name': 'A&W Restaurant',
  'location': {'address': '190 Barton St E',
   'country': 'CA',
   'cross_street': '',
   'formatted_address': '190 Barton St E, Hamilton ON L8L 2W8',
   'locality': 'Hamilton',
   'postcode': 'L8L 2W8',
   'region': 'ON'},
  'categories': ['American Restaurant', 'Fast Food Restaurant']},
 {'name': 'August 8',
  'location': {'address': '1 Wilson St',
   'address_extended': '# 12',
   'country': 'CA',
   'cross_street': 'James St.',
   'formatted_address': '1 Wilson St (James St.), Hamilton ON L8R 1C4',
   'locality': 'Hamilton',
   'postcode': 'L8R 1C4',
   'region': 'ON'},
  'categories': ['Dim Sum Restaurant', 'Sushi Restaurant']},
 {'name': 'The Burnt Tongue',
  'location': {'a

Put your parsed results into a DataFrame

In [217]:
# store results in dataframe
df = pd.DataFrame(results)
df

,name,location,categories
0,Bonanza Bakery,"{'address': '100 Murray St E', 'country': 'CA'...",[Bakery]
1,A&W Restaurant,"{'address': '190 Barton St E', 'country': 'CA'...","[American Restaurant, Fast Food Restaurant]"
2,August 8,"{'address': '1 Wilson St', 'address_extended':...","[Dim Sum Restaurant, Sushi Restaurant]"
3,The Burnt Tongue,"{'address': '10 Cannon St E', 'country': 'CA',...","[Fast Food Restaurant, Soup Spot]"
4,Mulberry Street Coffeehouse,"{'address': '193 James St N', 'country': 'CA',...","[Café, Coffee Shop, Restaurant]"
5,Charred,"{'address': '244 James St N', 'country': 'CA',...","[BBQ Joint, Portuguese Restaurant]"
6,Lulu's Shawarma,"{'address': '32 John St N', 'country': 'CA', '...","[Mediterranean Restaurant, Middle Eastern Rest..."
7,The Mule,"{'address': '41 King William St', 'country': '...",[Taco Restaurant]
8,Black Forest Inn,"{'address': '255 King St E', 'country': 'CA', ...",[German Restaurant]
9,Saltlick Smokehouse,"{'address': '282 James St N', 'country': 'CA',...",[BBQ Joint]


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating